# A7 : Support Vector Machines

---

## 1. the original formulation

In [1]:
import numpy as np
from cvxopt import matrix, solvers

# Data points
X = np.array([[0, 1], [-1, 0]])
y = np.array([1, -1])

# Dimensions
m, n = X.shape

# Construct the matrices for the QP problem
# P matrix for the quadratic part of the objective function (minimizing 0.5 * w.T * w)
P = matrix(np.eye(n + 1))  # Include the bias term in the weights matrix
q = matrix(
    np.zeros(n + 1)
)  # Coefficient for the linear part of the objective function (0 bcs only quadratic part)

# Append a column for the bias term, b
X_extended = np.hstack([X, np.ones((m, 1))])

# Construct G so that -y_i * (w . x_i + b) <= -1 (this means y_i * (w . x_i + b) >= 1)
G = matrix(-np.multiply(y[:, np.newaxis], X_extended))
h = matrix(-np.ones(m))  # The -1 on the right side of the inequalities

# Solve QP problem using CVXOPT
solution = solvers.qp(P, q, G, h)
w_b = np.array(solution["x"]).flatten()  # Retrieve the solution
w = w_b[:-1]
b = w_b[-1]

# Print the results (rounded to 6 decimal places)
print("Weights:", np.round(w, 6))  # Weight vector
print("Bias:", np.round(b, 6))  # Bias term

     pcost       dcost       gap    pres   dres
 0:  2.5000e-01  7.5000e-01  1e+00  1e+00  8e-17
 1:  5.8212e-01  9.4382e-01  2e-02  2e-01  2e-16
 2:  1.0018e+00  1.0000e+00  2e-03  7e-17  3e-15
 3:  1.0000e+00  1.0000e+00  2e-05  1e-17  6e-16
 4:  1.0000e+00  1.0000e+00  2e-07  1e-16  6e-16
Optimal solution found.
Weights: [1. 1.]
Bias: 0.0


---

## the dual form of the original formulation

In [2]:
import numpy as np
from cvxopt import matrix, solvers

# Training data
X = np.array([[0, 1], [-1, 0]])
y = np.array([1, -1])

# Dimensions
m, n = X.shape

# Kernel matrix calculation: y_i * y_j * x_i . x_j
# This represents the quadratic term in the dual problem's objective function
K = np.dot(X, X.T) * y[:, None] * y[None, :]

# Convert K into a cvxopt matrix of type 'd' (double precision floating point)
# P matrix: Represents the 0.5 * sum(alpha_i * alpha_j * y_i * y_j * x_i^T * x_j) part of the dual objective
P = matrix(K, tc="d")

# q vector: Represents the -sum(alpha_i) part of the dual objective
q = matrix(-np.ones(m), tc="d")

# G matrix: Sets the constraint that alpha_i >= 0 for all i (the inequality constraint)
G = matrix(-np.eye(m), tc="d")

# h vector: Corresponds to the non-negativity(0) constraint of alpha_i (alpha_i >= 0)
h = matrix(np.zeros(m), tc="d")

# A matrix: Ensures that the sum of alpha_i * y_i is 0 (the equality constraint)
A = matrix(y, (1, m), "d")

# b scalar: The constant zero in the equality constraint sum(alpha_i * y_i) = 0
b = matrix(0.0, tc="d")

# Solve QP problem using CVXOPT, which maximizes the dual objective function under the given constraints
solution = solvers.qp(P, q, G, h, A, b)
alphas = np.array(solution["x"]).flatten()

# Compute the weight vector w and the bias b from the solution
# Weights calculated as the sum of alpha_i * y_i * x_i, directly using the support vectors and their multipliers
w = np.sum(alphas * y[:, None] * X, axis=0)

# Find the indices of support vectors (alphas > very small threshold)
support_vectors = np.where(alphas > 1e-5)[0]

# Calculate the bias using one of the support vectors
# Bias is calculated to correct the hyperplane such that it correctly classifies at least one support vector
if support_vectors.size > 0:
    sv = support_vectors[0]
    b = y[sv] - np.dot(
        X[sv], w
    )  # Use the formula y_sv - (w . x_sv) to determine the correct offset
else:
    b = 0

# Print the results (rounded to 6 decimal places)
print("Weights:", np.round(w, 6))  # Weight vector that defines the hyperplane
print("Bias:", np.round(b, 6))  # Bias term that offsets the hyperplane

     pcost       dcost       gap    pres   dres
 0: -7.5000e-01 -1.7500e+00  1e+00  0e+00  1e+00
 1: -9.4382e-01 -9.5908e-01  2e-02  2e-16  2e-01
 2: -1.0000e+00 -1.0018e+00  2e-03  0e+00  6e-17
 3: -1.0000e+00 -1.0000e+00  2e-05  0e+00  2e-17
 4: -1.0000e+00 -1.0000e+00  2e-07  3e-16  9e-17
Optimal solution found.
Weights: [1. 1.]
Bias: -0.0
